In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import zipfile
import os

def extract_zip(input_zip, path_to_extract="./"):
    with zipfile.ZipFile(input_zip, 'r') as zip_ref:
        zip_info = zip_ref.infolist()
        for member in zip_info:
            try:
                member.filename = member.filename.encode('cp437').decode('utf-8')
            except:
                member.filename = member.filename.encode('cp437').decode('gbk')
            zip_ref.extract(member, path_to_extract)

zip_file_path = '/content/drive/MyDrive/datasets 3.zip'  # 更改為你的 zip 檔案名稱
extract_zip(zip_file_path)


In [18]:
# !rm -rf images

In [59]:
# Import necessary libraries
from torch.utils.data import DataLoader
# from torch.backends import cudnn  # Commented out, but available for use
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import os
from imutils import paths

# Print the availability of CUDA
print(torch.cuda.is_available())


True


In [60]:
# Define the character set for license plate recognition
CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
         '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
         '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
         '新', '拷', '貝',
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'I', 'O', '-']

In [61]:
# Import necessary libraries
import torch.nn as nn
import torch

# Define a small basic block module
class small_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(small_block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )

    def forward(self, x):
        return self.block(x)

# Define the LPRNet module
class LPRNet(nn.Module):
    def __init__(self, lpr_max_len, phase, class_num, dropout_rate):
        super(LPRNet, self).__init__()
        self.phase = phase
        self.lpr_max_len = lpr_max_len
        self.class_num = class_num

        # Define the backbone of the network
        self.backbone = nn.Sequential(  #                                                                 [batchsize,  3,24,94]
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),  # Convolution layer      [batchsize, 64,22,92] layer 0
            nn.BatchNorm2d(num_features=64),  # Batch normalization                                       [batchsize, 64,22,92] layer 1
            nn.ReLU(),  # Activation function                                                             [batchsize, 64,22,92] layer 2
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1)),  # Max pooling                         [batchsize, 64,20,90] layer 3
            small_block(ch_in=64, ch_out=128),  # Small basic block                                       [batchsize,128,20,90] layer 4
            nn.BatchNorm2d(num_features=128),  # Batch normalization                                      [batchsize,128,20,90] layer 5
            nn.ReLU(),  # Activation function                                                             [batchsize,128,20,90] layer 6
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2)),  # Max pooling                         [batchsize, 64,18,44] layer 7
            small_block(ch_in=64, ch_out=256),  # Small basic block                                       [batchsize,256,18,44] layer 8
            nn.BatchNorm2d(num_features=256),  # Batch normalization                                      [batchsize,256,18,44] layer 9
            nn.ReLU(),  # Activation function                                                             [batchsize,256,18,44] layer 10
            small_block(ch_in=256, ch_out=256),  # Small basic block                                      [batchsize,256,18,44] layer 11
            nn.BatchNorm2d(num_features=256),  # Batch normalization                                      [batchsize,256,18,44] layer 12
            nn.ReLU(),  # Activation function                                                             [batchsize,256,18,44] layer 13
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(4, 1, 2)),  # Max pooling                         [batchsize, 64,16,21] layer 14
            nn.Dropout(dropout_rate),  # Dropout to prevent overfitting                                   [batchsize, 64,16,21] layer 15
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(1, 4), stride=1),# Convolution layer [batchsize,256,16,18] layer 16
            nn.BatchNorm2d(num_features=256),  # Batch normalization                                      [batchsize,256,16,18] layer 17
            nn.ReLU(),  # Activation function                                                             [batchsize,256,16,18] layer 18
            nn.Dropout(dropout_rate),  # Dropout to prevent overfitting                                   [batchsize,256,16,18] layer 19
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1),  # Convolution layer to predict class, classnum = 68
            nn.BatchNorm2d(num_features=class_num),  # Batch normalization                                [batchsize, 68, 4,18] layer 21
            nn.ReLU(),  # Activation function                                                             [batchsize, 68, 4,18] layer 22
        )

        # Container to combine features
        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
            # Additional layers can be added here if necessary
        )

    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            x = layer(x)
            if i in [2, 6, 13, 22]:  # Select specific layers' outputs to keep
                keep_features.append(x)
        # keep_features:
        #  [batchsize, 64,22,92] layer 2
        #  [batchsize,128,20,90] layer 6
        #  [batchsize,256,18,44] layer 13
        #  [batchsize, 68, 4,18] layer 22

        global_context = list()
        # Process and combine features from different layers
        for i, f in enumerate(keep_features):
            if i == 0:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i == 1:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i == 2:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)

            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            global_context.append(f)
        #keep_features after process:
        #  [batchsize, 64, 4,18] layer 2
        #  [batchsize,128, 4,18] layer 6
        #  [batchsize,256, 4,18] layer 13
        #  [batchsize, 68, 4,18] layer 22

        x = torch.cat(global_context, 1)  # Combine context features
        x = self.container(x)  # Pass through container
        logits = torch.mean(x, dim=2)  # Aggregate features

        return logits # [batchsize, 68, 18]


In [62]:
# Import necessary libraries
from imutils import paths
import numpy as np
import random
import cv2
import os
from torch.utils.data import Dataset

# Create a dictionary for character to index mapping
CHARS_DICT = {char: i for i, char in enumerate(CHARS)}

# Define the dataset loader class for LPR
class LPRDataLoader(Dataset):
    def __init__(self, img_dir, imgSize, lpr_max_len, PreprocFun=None):
        self.img_dir = img_dir  # Directory containing images

        self.img_paths = []  # List to store image paths

        pathsImg = os.listdir(img_dir)  # List images in directory
        for im in pathsImg:
            self.img_paths.append(os.path.join(img_dir, im))

        random.shuffle(self.img_paths)  # Shuffle image paths


        self.img_size = imgSize  # Target image size
        self.lpr_max_len = lpr_max_len  # Max length of license plate

        if PreprocFun is not None:
            self.PreprocFun = PreprocFun
        else:
            self.PreprocFun = self.transform

    def __len__(self):
        return len(self.img_paths)  # Return the total number of images

    def __getitem__(self, index):
        filename = self.img_paths[index]  # Get filename at the specified index
        npfilename = np.fromfile(filename, dtype=np.uint8) # filename to np
        Image = cv2.imdecode(npfilename, -1)  # Read image

        try:
            Image = cv2.cvtColor(Image, cv2.COLOR_RGB2BGR)
        except:
            filename = self.last_filename  # Get filename at the specified index
            npfilename = np.fromfile(filename, dtype=np.uint8) # filename to np
            Image = cv2.imdecode(npfilename, -1)  # Read image

        height, width, _ = Image.shape
        if height != self.img_size[1] or width != self.img_size[0]:
            Image = cv2.resize(Image, self.img_size)  # Resize image
        Image = self.PreprocFun(Image)  # Preprocess image

        basename = os.path.basename(filename)
        imgname, suffix = os.path.splitext(basename)
        imgname = imgname.split("-")[0].split("_")[0]  # Extract the label from filename
        label = list()
        for c in imgname:
           label.append(CHARS_DICT[c]) # Convert label into indices
        self.last_filename = filename

        return Image, label, len(label)  # Return processed image, label, and label length

    def transform(self, img):
        img = img.astype('float32')
        img -= 127.5  # Zero-center
        img *= 0.0078125  # Normalize
        img = np.transpose(img, (2, 0, 1))  # Change channel order

        return img  # Return transformed image

    def check(self, label):
        # Check if the label meets a specific condition
        if label[2] != CHARS_DICT['D'] and label[2] != CHARS_DICT['F'] \
                and label[-1] != CHARS_DICT['D'] and label[-1] != CHARS_DICT['F']:
            print("Error label, Please check!")  # Print error if condition is not met
            return False
        else:
            return True  # Return True if the label is correct


In [63]:
def sparse_tuple_for_ctc(T_length, lengths):
    """
    Counts the tuples of input lengths and target lengths

    Args:
        optimizer: The total length T_length for each input
        cur_epoch: The actual length of each target sequence

    Returns:
        -Tuple of input lengths
        -Tuple of target lengths
    """
    # Initialize lists to hold input and target lengths
    input_lengths = []
    target_lengths = []

    # Iterate over the provided lengths
    for ch in lengths:
        input_lengths.append(T_length)  # Append the total length T_length for each input
        target_lengths.append(ch)  # Append the actual length of each target sequence

    # Return tuples of input lengths and target lengths
    return tuple(input_lengths), tuple(target_lengths)


In [65]:
def adjust_learning_rate(optimizer, cur_epoch, base_lr, lr_schedule):
    """
    Adjusts the learning rate based on the current epoch and a predefined learning rate schedule.

    Args:
        optimizer: The optimizer for which to adjust the learning rate.
        cur_epoch: The current epoch number.
        base_lr: The base learning rate before applying the schedule.
        lr_schedule: A list of epochs at which the learning rate should be decreased.

    Returns:
        The new learning rate.
    """

    # Initialize the learning rate to 0
    lr = 0

    # Iterate over the learning rate schedule
    for i, e in enumerate(lr_schedule):
        # If the current epoch is before the next scheduled decrease, update the learning rate
        if cur_epoch < e:
            lr = base_lr * (0.1 ** i)  # Decrease the learning rate
            break  # Exit the loop once the new rate is set

    # If the learning rate was not updated by the schedule, use the base learning rate
    if lr == 0:
        lr = base_lr

    # Apply the new learning rate to the optimizer
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # Return the new learning rate
    return lr


In [114]:
def get_parser():
    """
    Creates and returns an argument parser for command-line options.

    Returns:
        The ArgumentParser object with train net parameters.
    """
    # Initialize the argument parser
    parser = argparse.ArgumentParser(description='parameters to train net')

    # Add arguments to the parser
    parser.add_argument('--max_epoch', default=240, help='epoch to train the network')
    parser.add_argument('--img_size', default=[94, 24], help='the image size')
    parser.add_argument('--train_img_dirs', default="./datasets/images/train", help='the train images path')
    parser.add_argument('--test_img_dirs', default="./datasets/images/val", help='the test images path')
    parser.add_argument('--dropout_rate', default=0.5, help='dropout rate.')
    parser.add_argument('--learning_rate', default=0.01, help='base value of learning rate.')
    parser.add_argument('--lpr_max_len', default=8, help='license plate number max length.')
    parser.add_argument('--train_batch_size', default=128, help='training batch size.')
    parser.add_argument('--test_batch_size', default=128, help='testing batch size.')
    parser.add_argument('--phase_train', default=True, type=bool, help='train or test phase flag.')
    parser.add_argument('--num_workers', default=0, type=int, help='Number of workers used in dataloading')
    parser.add_argument('--cuda', default=True, type=bool, help='Use cuda to train model')
    parser.add_argument('--resume_epoch', default=0, type=int, help='resume iter for retraining')
    parser.add_argument('--save_interval', default=500, type=int, help='interval for save model state dict')
    parser.add_argument('--test_interval', default=100, type=int, help='interval for evaluate')
    parser.add_argument('--momentum', default=0.88, type=float, help='momentum')
    parser.add_argument('--weight_decay', default=2e-5, type=float, help='Weight decay for SGD')
    parser.add_argument('--lr_schedule', default=[160, 200, 220, 230, 240], help='schedule for learning rate.')
    parser.add_argument('--save_folder', default=r'./final2', help='Location to save checkpoint models')
    parser.add_argument('--pretrained_model', default='', help='pretrained model path (optional)')
    parser.add_argument('-f', type=str, default="read parameters", help='Dummy argument for Jupyter Notebooks')

    # Parse and return the arguments
    args = parser.parse_args()
    return args


In [68]:
def collate_fn(batch):
    """
    Custom collate function for data loading.

    This function prepares a batch for training by aggregating the images, labels, and lengths
    from individual samples into collective batches for processing.

    Args:
        batch: A list of tuples where each tuple corresponds to a single sample. Each tuple contains
               - img: The image data as a numpy array.
               - label: The label of the image.
               - length: The length of the label.

    Returns:
        A tuple containing:
        - A tensor of stacked images.
        - A tensor of concatenated labels.
        - A list of lengths of each label in the batch.
    """
    imgs = []    # List to store image tensors
    labels = []  # List to store all labels
    lengths = [] # List to store lengths of the labels

    # Loop over every sample in the batch
    for _, sample in enumerate(batch):
        img, label, length = sample  # Unpack the sample

        imgs.append(torch.from_numpy(img))  # Convert image to tensor and append
        labels.extend(label)  # Extend the labels list with the label from the sample
        lengths.append(length)  # Append the length of the label

    # Convert labels list to a numpy array, flatten it, and then convert to int
    labels = np.asarray(labels).flatten().astype(int)

    # Return a tuple of stacked image tensors, labels tensor, and lengths list
    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)


In [115]:
def train():
    args = get_parser()  # Parse arguments from command line

    T_length = 18  # Placeholder for args.lpr_max_len, typically the maximum length of license plate
    epoch = 0 + args.resume_epoch  # Starting epoch
    loss_val = 0  # Initialize loss value

    # Create save folder if it doesn't exist
    if not os.path.exists(args.save_folder):
        os.makedirs(args.save_folder)

    # Initialize the LPRNet model with parameters
    lprnet = LPRNet(lpr_max_len=args.lpr_max_len, phase=args.phase_train, class_num=len(CHARS), dropout_rate=args.dropout_rate)
    device = torch.device("cuda:0" if args.cuda else "cpu")  # Define the device
    lprnet.to(device)  # Move the model to the device
    print("Successfully build network!")

    # Load a pretrained model if specified
    if args.pretrained_model:
        lprnet.load_state_dict(torch.load(args.pretrained_model))
        print("Successfully load pretrained model!")
    else:
        # If no pretrained model is specified, initialize weights
        def weights_init(m):
            for key in m.state_dict():
                if key.split('.')[-1] == 'weight':
                    if 'conv' in key:
                        nn.init.kaiming_normal_(m.state_dict()[key], mode='fan_out')
                    if 'bn' in key:
                        nn.init.constant_(m.state_dict()[key], 1)
                elif key.split('.')[-1] == 'bias':
                    nn.init.constant_(m.state_dict()[key], 0.01)
        # Apply the weights initialization
        lprnet.backbone.apply(weights_init)
        lprnet.container.apply(weights_init)
        print("Successfully initialized network weights!")

    # Define the optimizer (RMSprop is used here)
    optimizer = optim.RMSprop(lprnet.parameters(), lr=args.learning_rate, alpha=0.9, eps=1e-08, momentum=args.momentum, weight_decay=args.weight_decay)

    # Paths for training and testing datasets
    train_img_dirs = os.path.expanduser(args.train_img_dirs)
    test_img_dirs = os.path.expanduser(args.test_img_dirs)

    # Load datasets
    train_dataset = LPRDataLoader(train_img_dirs, args.img_size, args.lpr_max_len)
    test_dataset = LPRDataLoader(test_img_dirs, args.img_size, args.lpr_max_len)

    # Calculate the number of iterations per epoch and the total number of iterations
    epoch_size = len(train_dataset) // args.train_batch_size
    max_iter = args.max_epoch * epoch_size

    # Define the CTC Loss
    ctc_loss = nn.CTCLoss(blank=len(CHARS)-1, reduction='mean')

    # Determine the starting iteration based on whether training is resumed
    start_iter = args.resume_epoch * epoch_size if args.resume_epoch > 0 else 0
    acc = 0
    # Training loop
    for iteration in range(start_iter, max_iter):
        if iteration % epoch_size == 0:
            # At the start of each epoch, create a batch iterator
            batch_iterator = iter(DataLoader(train_dataset, args.train_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))
            loss_val = 0
            epoch += 1

        # Save the model at specified intervals
        if iteration != 0 and iteration % args.save_interval == 0:
            torch.save(lprnet.state_dict(), os.path.join(args.save_folder, 'LPRNet_' + '_iteration_' + str(iteration) + '.pth'))

        # Perform evaluation at specified intervals
        if (iteration + 1) % args.test_interval == 0:
            _acc = Greedy_Decode_Eval(lprnet, test_dataset, args)
            if _acc > acc:
                torch.save(lprnet.state_dict(), os.path.join(args.save_folder, 'LPRNet_best' + str(_acc) + '.pth'))
                acc = _acc
                print("saved")
            # Note: Should switch back to train mode after evaluation

        start_time = time.time()
        images, labels, lengths = next(batch_iterator)  # Load a batch of training data

        # Prepare data for CTC Loss
        input_lengths, target_lengths = sparse_tuple_for_ctc(T_length, lengths)

        # Adjust learning rate based on the schedule
        lr = adjust_learning_rate(optimizer, epoch, args.learning_rate, args.lr_schedule)

        # Move data to the device
        images = Variable(images, requires_grad=False).to(device)
        labels = Variable(labels, requires_grad=False).to(device)

        # Forward pass
        logits = lprnet(images)
        log_probs = logits.permute(2, 0, 1).log_softmax(2).requires_grad_()  # Prepare logits for CTC Loss

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss = ctc_loss(log_probs, labels, input_lengths=input_lengths, target_lengths=target_lengths)
        if loss.item() != np.inf:  # Ignore infinite loss values
            loss.backward()
            optimizer.step()

        loss_val += loss.item()
        end_time = time.time()

        # Logging
        if iteration % 20 == 0:
            print('Epoch:{0} || epochiter: {1}/{2} || Totel iter {3} || Loss: {4:.4f} ||'.format(epoch, iteration % epoch_size, epoch_size, iteration, loss.item()) +
                  ' Batch time: {0:.4f} sec. ||'.format(end_time - start_time) + ' LR: {0:.8f}'.format(lr))

    # Save the final model weights
    torch.save(lprnet.state_dict(), os.path.join(args.save_folder, 'lprnet-final.pth'))


In [116]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import time

def Greedy_Decode_Eval(Net, datasets, args):
    """
    Evaluate the network using a greedy decoding strategy.

    Args:
        Net: The trained network model for evaluation.
        datasets: The dataset to evaluate the network on.
        args: Command-line arguments including cuda usage and batch size settings.

    This function computes the accuracy of the network on the provided dataset by comparing
    the predicted labels with the true labels. It uses a greedy decoding strategy for predictions.
    """
    # Calculate the number of batches based on the dataset size and batch size
    epoch_size = len(datasets) // args.test_batch_size
    # Create a batch iterator
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))

    # Initialize counters for true positives, type 1 and type 2 errors
    Tp = 0
    Tn_1 = 0
    Tn_2 = 0

    # Record start time for speed measurement
    t1 = time.time()

    # Loop through the dataset
    for _ in range(epoch_size):
        images, labels, lengths = next(batch_iterator)  # Load a batch of data

        # Convert labels into a list of numpy arrays for easy comparison later
        start = 0
        targets = []
        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])

        # Move images to GPU if CUDA is enabled
        images = Variable(images.cuda()) if args.cuda else Variable(images)

        # Forward pass to get predictions
        prebs = Net(images).cpu().detach().numpy()

        # Greedy decoding: select the label with the highest probability at each step
        preb_labels = []
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]
            preb_label = [np.argmax(preb[:, j], axis=0) for j in range(preb.shape[1])]

            # Remove repeated labels and blanks
            no_repeat_blank_label = []
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:  # Check if not a blank label
                no_repeat_blank_label.append(pre_c)
            for c in preb_label: # dropout repeate label and blank label
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)

        # Compare predicted labels with true labels and update counters
        for i, label in enumerate(preb_labels):
            if len(label) != len(targets[i]):
                Tn_1 += 1  # Prediction and target lengths do not match
                continue
            if np.array_equal(np.asarray(targets[i]), np.asarray(label)):
                Tp += 1  # Prediction matches target
            else:
                Tn_2 += 1  # Prediction does not match target

    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
    print("[Info] Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
    t2 = time.time()
    print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))
    return Acc


In [117]:
if __name__ == "__main__":
    train()

Successfully build network!
Successfully initialized network weights!
Epoch:1 || epochiter: 0/24 || Totel iter 0 || Loss: 7.9954 || Batch time: 0.0597 sec. || LR: 0.01000000
Epoch:1 || epochiter: 20/24 || Totel iter 20 || Loss: 6.9137 || Batch time: 0.0549 sec. || LR: 0.01000000
Epoch:2 || epochiter: 16/24 || Totel iter 40 || Loss: 4.0181 || Batch time: 0.0533 sec. || LR: 0.01000000
Epoch:3 || epochiter: 12/24 || Totel iter 60 || Loss: 3.2006 || Batch time: 0.0606 sec. || LR: 0.01000000
Epoch:4 || epochiter: 8/24 || Totel iter 80 || Loss: 2.4852 || Batch time: 0.0534 sec. || LR: 0.01000000
[Info] Test Accuracy: 0.0 [0:607:33:640]
[Info] Test Speed: 0.00026730123838242383s 1/701]
Epoch:5 || epochiter: 4/24 || Totel iter 100 || Loss: 1.5692 || Batch time: 0.0529 sec. || LR: 0.01000000
Epoch:6 || epochiter: 0/24 || Totel iter 120 || Loss: 1.0759 || Batch time: 0.0545 sec. || LR: 0.01000000
Epoch:6 || epochiter: 20/24 || Totel iter 140 || Loss: 0.7676 || Batch time: 0.0544 sec. || LR: 0.01

In [123]:
import argparse
import os
import time
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from PIL import Image, ImageDraw, ImageFont
from google.colab.patches import cv2_imshow


def get_parser_test():
    parser = argparse.ArgumentParser(description='parameters to test net')
    parser.add_argument('--img_size', default=[94, 24], help='the image size')
    parser.add_argument('--test_img_dirs', default=r"./datasets/images/test", help='the test images path')
    parser.add_argument('--dropout_rate', default=0, help='dropout rate.')
    parser.add_argument('--lpr_max_len', default=8, help='license plate number max length.')
    parser.add_argument('--test_batch_size', default=100, help='testing batch size.')
    parser.add_argument('--phase_train', default=False, type=bool, help='train or test phase flag.')
    parser.add_argument('--num_workers', default=0, type=int, help='Number of workers used in dataloading')
    parser.add_argument('--cuda', default=True, type=bool, help='Use cuda to train model')
    parser.add_argument('--show', default=True, type=bool, help='show test image and its predict result or not.')
    parser.add_argument('--pretrained_model', default=r'/content/final2/LPRNet_best0.7921875.pth', help='pretrained base model')
    parser.add_argument('-f', type=str, default="read parameters", help='Dummy argument for Jupyter Notebooks')

    args = parser.parse_args()

    return args

# Custom collate function for data loader
def collate_fn(batch):
    imgs, labels, lengths = [], [], []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

# Function to test the network
def test():

    args = get_parser_test()

    # Initialize the network and move it to the appropriate device
    lprnet = LPRNet(lpr_max_len=args.lpr_max_len, phase=args.phase_train, class_num=len(CHARS), dropout_rate=args.dropout_rate)
    device = torch.device("cuda:0" if args.cuda else "cpu")
    lprnet.to(device)
    print("Successful to build network!")
    if args.pretrained_model:
        lprnet.load_state_dict(torch.load(args.pretrained_model))
        print("Load pretrained model successful!")

    # Prepare the test dataset and perform evaluation
    test_img_dirs = os.path.expanduser(args.test_img_dirs)
    test_dataset = LPRDataLoader(test_img_dirs, args.img_size, args.lpr_max_len)
    try:
        acc = Greedy_Decode_Eval(lprnet, test_dataset, args)
    finally:
        cv2.destroyAllWindows()
def Greedy_Decode_Eval(Net, datasets, args):
    # TestNet = Net.eval()
    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))

    Tp = 0
    Tn_1 = 0
    Tn_2 = 0
    t1 = time.time()
    for i in range(epoch_size):
        # load train data
        images, labels, lengths = next(batch_iterator)
        start = 0
        targets = []
        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])
        imgs = images.numpy().copy()

        if args.cuda:
            images = Variable(images.cuda())
        else:
            images = Variable(images)

        # forward
        # images: [bs, 3, 24, 94]
        # prebs:  [bs, 68, 18]
        prebs = Net(images)
        # greedy decode
        prebs = prebs.cpu().detach().numpy()
        preb_labels = list()
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]  # all images [68, 18]
            preb_label = list()
            for j in range(preb.shape[1]):
                preb_label.append(np.argmax(preb[:, j], axis=0))
            no_repeat_blank_label = list()
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:  # record duplicate characters
                no_repeat_blank_label.append(pre_c)
            for c in preb_label:  # delete'-'
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)
        for i, label in enumerate(preb_labels):  # calculate accuracy
            # show image and its predict label
            if args.show and i % 10 == 0:
                try:
                    show(imgs[i], label, targets[i])
                except:
                    pass
            if len(label) != len(targets[i]):
                Tn_1 += 1  # error+1
                continue
            if (np.asarray(targets[i]) == np.asarray(label)).all():
                Tp += 1  # correct+1
            else:
                Tn_2 += 1
    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
    print("[Info] Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
    t2 = time.time()
    print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))
    return Acc
# Function to show the image, its predicted label, and the actual label
def show(img, label, target):
    img = np.transpose(img, (1, 2, 0))
    img *= 128.
    img += 127.5
    img = img.astype(np.uint8)

    lb = "".join([CHARS[i] for i in label])
    tg = "".join([CHARS[int(j)] for j in target.tolist()])

    flag = "T" if lb == tg else "F"
    # img = cv2ImgAddText(img, lb, (0, 0))
    cv2_imshow(img)
    print(f"target: {tg}, ### {flag} ###, predict: {lb}")
    cv2.waitKey()
    cv2.destroyAllWindows()

# Add text to an image using PIL for better font handling
def cv2ImgAddText(img, text, pos, textColor=(255, 0, 0), textSize=12):
    if isinstance(img, np.ndarray):  # Convert from OpenCV to PIL format
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype("./NotoSansCJK-Regular.ttc", textSize, encoding="utf-8")
    draw.text(pos, text, textColor, font=fontText)

    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)


In [133]:
if __name__ == "__main__":
    test()

Successful to build network!
Load pretrained model successful!


target: 皖AD04030, ### T ###, predict: 皖AD04030


target: 皖AD09556, ### T ###, predict: 皖AD09556


target: 皖AD09266, ### F ###, predict: 皖AD057


target: 皖AD08379, ### T ###, predict: 皖AD08379


...

target: 皖AD97689, ### T ###, predict: 皖AD97689


target: 皖AD03699, ### T ###, predict: 皖AD03699


target: 皖AD07168, ### T ###, predict: 皖AD07168


target: 皖AD99929, ### T ###, predict: 皖AD99929


target: 皖AD00390, ### T ###, predict: 皖AD00390
[Info] Test Accuracy: 0.84 [588:42:70:700]
[Info] Test Speed: 0.0009792973413617057s 1/701]
